# Dashboard for the Argentinian Properties
In this project, we revisit the [project on Argentinian Property listings](https://github.com/Mathhead/Sample_Projects/blob/master/argentinian_property_listings.ipynb) and build a simple dashboard to visualize the data.

In [1]:
import numpy as np

np.random.seed( 1234567890 )

print( np.__version__ )

1.16.2


In [2]:
import pandas as pd

print( pd.__version__ )

0.24.2


In [3]:
property_data = pd.read_csv( "C:\\Users\\rhopi\\OneDrive\\Desktop\\kaggle data 1\\property listings data set\\ar_properties.csv" )

property_data.head()

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,7LoZFkSIPOTox0r32ck42Q==,Propiedad,2019-09-15,2019-10-26,2019-09-15,-26.815439,-65.312393,Argentina,Tucumán,Yerba Buena,...,NaN,NaN,NaN,NaN,NaN,Mensual,Terreno - Yerba Buena,Accesos pavimentados.<br>Red subterránea de ag...,Lote,Venta
1,QsZD4OxZInNd5po5LQDRmg==,Propiedad,2019-09-15,9999-12-31,2019-09-15,-26.839469,-65.212790,Argentina,Tucumán,NaN,...,1.0,55.0,41.0,NaN,NaN,Mensual,Departamento - Capital,Detalles constructivos:<br><br>Pisos de porcel...,Departamento,Venta
2,qDtysoUgbnHLp0W9We+8fg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.424820,-64.181225,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | San Lorenzo al 400,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
3,olj51zV0HFurmaZ78U0ssg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.421242,-64.190798,Argentina,Córdoba,Córdoba,...,1.0,48.0,48.0,NaN,NaN,Mensual,1 DORMITORIO | M. T. de Alvear al 500,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal
4,OcS1SAA5oAzjZ3Mzg3XHyg==,Propiedad,2019-09-15,2019-09-25,2019-09-15,-31.426064,-64.180042,Argentina,Córdoba,Córdoba,...,1.0,45.0,45.0,NaN,NaN,Mensual,1 DORMITORIO | Av. Poeta Lugones al 200,Alquiler temporario de Departamento 1 DORMITOR...,Departamento,Alquiler temporal


Here, we build only a simple model, that focuses on __lat__, __lon__, __bathrooms__, __surface_total__ to predict __price__.<br>
We only consider "Departamentos" which are "Venta" and which are prized in USD.

In [4]:
property_data = property_data.dropna( subset = [ "price", "lon", "lat", "bathrooms", "surface_total" ] )

In [5]:
property_data = property_data[ property_data[ "currency" ] == "USD" ]

property_data = property_data[ property_data[ "property_type" ] == "Departamento" ]

property_data = property_data[ property_data[ "operation_type" ] == "Venta" ]

We drop all columns but the aforementioned.

In [6]:
for column in property_data.columns:
    
    if column not in [ "lat", "lon", "bathrooms", "surface_total", "price" ]:
        
        property_data = property_data.drop( [ column ], axis = 1 )
        
property_data.head()

,lat,lon,bathrooms,surface_total,price
61,-34.572445,-58.420624,3.0,193.0,935000.0
63,-34.586423,-58.414190,1.0,32.0,132900.0
64,-34.584758,-58.413319,1.0,88.0,230000.0
65,-34.578491,-58.431951,1.0,36.0,108000.0
68,-34.561679,-58.458689,1.0,80.0,269000.0


In [7]:
print( property_data.isna().sum() )

lat              0
lon              0
bathrooms        0
surface_total    0
price            0
dtype: int64


We now use dash to build a web-app to visualize the data.

In [8]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

print( dash.__version__ )

1.14.0


In [9]:
import jupyter_dash
from jupyter_dash import JupyterDash

print( jupyter_dash.__version__ )

0.3.1


In [10]:
import plotly
import plotly.graph_objs as go

print( plotly.__version__ )

4.9.0


In [11]:
app_scatter = JupyterDash( __name__ )

In [12]:
app_scatter.css.append_css( { "external_url" : "https://codepen.io/chriddyp/pen/bWLwgP.css" } )

In [13]:
features = [ "Latitude", "Longitude", "Bathrooms", "Surface total", "Price" ]

dropdown = [ { "label" : x, "value" : x } for x in features ]

In [14]:
dropdown_x_var = dcc.Dropdown( id = "x-var", options = dropdown, value = "Longitude" )

div_x_var = html.Div( children = [ html.H4( "Variable for x Axis: " ), dropdown_x_var ], className = "six columns" )

In [15]:
dropdown_y_var = dcc.Dropdown( id = "y-var", options = dropdown, value = "Latitude" )

div_y_var = html.Div( children = [ html.H4( "Variable for y Axis: " ), dropdown_y_var ], className = "six columns" )

In [16]:
dict_ = { "Longitude" : "lon", "Latitude" : "lat", "Bathrooms" : "bathrooms", "Surface total" : "surface_total",
        "Price" : "price" }

In [17]:
app_scatter.layout = html.Div( children = [ html.H1( "Scatterplot of Argentinian Property Listings" ),
                                  html.H2( "Interactive Scatterplot" ), html.Div( children = [ div_x_var, div_y_var ],
                                                                                className = "row" ), 
                                   dcc.Graph( id = "scatter" ) ] )

In [18]:
@app_scatter.callback( Output( component_id = "scatter", component_property = "figure" ), 
             [ Input( component_id = "x-var", component_property = "value" ), 
              Input( component_id = "y-var", component_property = "value" ) ] )

def scatter_plot( x_col, y_col ):
    """
        we begin with sampling 2000 data points. If we do not sample, the plotting function may get overwhelmed.
    """
    
    data = property_data.sample( n = 2000 )
    
    trace = go.Scatter( x = data[ dict_[ x_col ] ], y = data[ dict_[ y_col ] ], mode = "markers" )
    
    layout = go.Layout( title = "Scatterplot", xaxis = dict( title = x_col ), yaxis = dict( title = y_col ) )
    
    output_plot = go.Figure( data = [ trace ], layout = layout )
    
    return output_plot

In [19]:
app_scatter.run_server( mode = "inline" )

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.



C:\Users\rhopi\Anaconda3\lib\site-packages\dash\resources.py:68: UserWarning: You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources
  ).format(s["external_url"])
